In [1]:
import pandas as pd
from process import run_ols
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from scipy.stats import norm
from MLE import MLE



In [2]:
players = [] # array of dataframes where players[i] is the i+1th player's data
for i in range(1000):
    file = "../data/player_data/student_" + str(i + 1) + ".csv"
    players.append(pd.read_csv(file))

In [3]:
total_score = np.zeros((3, 1000))
num_score = np.zeros((3, 1000))

for i in range(1000):
    player = players[i]
    for _, row in player.iterrows():
        if row["game_scheduled"] == "morning":
            total_score[0][i] += row["score"]
            num_score[0][i] += 1
        elif row["game_scheduled"] == "noon":
            total_score[1][i] += row["score"]
            num_score[1][i] += 1
        elif row["game_scheduled"] == "evening":
            total_score[2][i] += row["score"]
            num_score[2][i] += 1

avg_score = np.divide(total_score, num_score, out=np.zeros_like(total_score), where=num_score!=0)

In [4]:
# avg_score[0] = "m", avg_score[1] = "n", avg_score[2] = "e"
expected_score = np.sum(avg_score, axis=0) / 3

In [5]:
# determine f(x, y)
mle = MLE("../data/previous_season_results.csv")
sigma = mle.calculate_MLE() # obtain from MLE
print(sigma)
def winProb(score_1, score_2):
    return 1 - norm(loc = score_1 - score_2 , scale = sigma).cdf(0)

10.828673870065314


In [18]:
# to maximize E[win], we should choose the 10 highest players
top_athletes = np.argpartition(expected_score, -10)[-10:] + 1
print("these are our top athletes:", top_athletes)

# team score
top_scores = expected_score[top_athletes - 1]
top_team_score = np.average(top_scores)
print("our team score is: ", top_team_score)

# expected number of wins
matchup = pd.read_csv("../data/season_match_up.csv")
matchup = matchup.to_numpy()[0, 1:] # 20 games
E_win = np.sum([winProb(top_team_score, matchup_score) for matchup_score in matchup])
print("Expected number of wins: ", E_win)

these are our top athletes: [268 725 790  89 138 791 631 785 521 129]
our team score is:  75.33198809523809
Expected number of wins:  11.375058670272534
